## Testing different cascades
Below we have functions for 2 tests, one on captured video from webcam and one on test images in *AWEForSegmentation*
dataset.


In [26]:
import cv2
import os
import numpy as np
import pandas as pd


def camera_test(cascade_folder):
    """Shows recognized ears with cascade from 'cascade_folder' on video from webcam."""
    # Load the cascade
    ear_cascade = cv2.CascadeClassifier(f'{cascade_folder}/cascade.xml')

    # To capture video from webcam.
    cap = cv2.VideoCapture(0)

    while True:
        # Read the frame
        _, img = cap.read()
        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Detect the faces
        ears = ear_cascade.detectMultiScale(img, 1.1, 100)
        for (x, y, w, h) in ears:
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)
        # Draw the rectangle around each ear
        # Display
        cv2.imshow('img', img)
        # Stop if escape key is pressed
        k = cv2.waitKey(30) & 0xff
        if k==27:
            break
    # Release the VideoCapture object
    cap.release()


def awe_test(cascade_folder, scaleFactor, minNeighbours, test_visual_folder=None):
    """Tests cascade from 'cascade_folder' using parameters 'scaleFactor' and 'minNeighbours'.
    If test_visual_folder is provided """
    results = {'avgIoU': 0, 'TP': 0, 'FP': 0}     # results will be averaged over all test images
    ear_cascade = cv2.CascadeClassifier(f'{cascade_folder}/cascade.xml')
    if test_visual_folder:
        if not os.path.exists(test_visual_folder):
                os.makedirs(test_visual_folder)
        # else:
        #     for file in os.listdir(f'{test_visual_folder}/{cascade_folder}'):
        #         os.remove(f'{test_visual_folder}/{cascade_folder}/{file}')
    n = 0
    for photo in os.listdir('AWEForSegmentation/test'):
        mask = cv2.imread(f'./AWEForSegmentation/testannot_rect/{photo}', 0) / 255
        img = cv2.imread(f'./AWEForSegmentation/test/{photo}', cv2.IMREAD_UNCHANGED)
        ears = ear_cascade.detectMultiScale(img, scaleFactor, minNeighbours)
        avg_iou = 0
        for (x, y, w, h) in ears:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)  # Draw the rectangle around each ear
            intersection = mask[y:y+h, x:x+h]
            union = mask.copy()
            union[y:y+h, x:x+h] = 1
            iou = np.sum(intersection) / np.sum(union)
            avg_iou += iou
            n += 1
            if iou > 0.5:
                results['TP'] += 1
            else:
                results['FP'] += 1
        if len(ears):
            results['avgIoU'] += avg_iou
        # if '3gfresw' in photo and cascade_folder != 'cascade':  # False
        # # if True:
        #     cv2.imshow('img', img)  # Display
        #     cv2.waitKey(0)
        if test_visual_folder:
            cv2.imwrite(f'{test_visual_folder}/{photo[:4]}_{cascade_folder}.png', img)
    results['avgIoU'] /= n
    return results

In [7]:
camera_test('cascadecoco_people')

In [28]:
def count_ears_test():
    nr_ears = 0
    for photo in os.listdir('AWEForSegmentation/test'):
        mask = cv2.imread(f'./AWEForSegmentation/testannot_rect/{photo}', 0).astype(bool)
        i = 0
        while i < mask.shape[0]:
            j = 0
            while j < mask.shape[1]:
                if mask[i, j]:
                    if i == 0 or not mask[i-1, j]:                  # upper row
                        if j == 0 or not mask[i, j-1]:              # upper left corner
                            nr_ears += 1
                j += 1
            i += 1
    return nr_ears

count_ears_test()

288

In [27]:
cascades = ['cascade', 'cascadegs', 'cascadecoco', 'cascadecoco_people']
results = pd.DataFrame(columns=cascades)
for cascade_folder in cascades:
    results[cascade_folder] = pd.Series(awe_test(cascade_folder, 1.1, 40))

1065
1114
1440
642


In [25]:
cascades = ['cascade', 'cascadegs', 'cascadecoco', 'cascadecoco_people']
results2 = pd.DataFrame(columns=cascades)
for cascade_folder in cascades:
    results2[cascade_folder] = pd.Series(awe_test(cascade_folder, 1.1, 5))

In [21]:
results3 = pd.DataFrame(columns=cascades)
for cascade_folder in cascades:
    results3[cascade_folder] = pd.Series(awe_test(cascade_folder, 1.1, 100))

In [17]:
cascades = ['cascade', 'cascadegs', 'cascadecoco', 'cascadecoco_people']
results4 = pd.DataFrame(columns=cascades)
for cascade_folder in cascades:
    results4[cascade_folder] = pd.Series(awe_test(cascade_folder, 1.1, 50, 'test_vis'))

In [18]:
cascades = ['cascade', 'cascadegs', 'cascadecoco', 'cascadecoco_people']
results5 = pd.DataFrame(columns=cascades)
for cascade_folder in cascades:
    results5[cascade_folder] = pd.Series(awe_test(cascade_folder, 1.1, 100, 'test_vis_100'))


In [19]:
cascades = ['cascade', 'cascadegs', 'cascadecoco', 'cascadecoco_people']
results5 = pd.DataFrame(columns=cascades)
for cascade_folder in cascades:
    results5[cascade_folder] = pd.Series(awe_test(cascade_folder, 1.1, 30, 'test_vis_30'))

